This entire script is dedicated to running at a single, base temperature of 1.2K and power sweeps 

Laser power can vary from -15dBm to +13dBm

Notes

- Do not blindly hit run all; ensure that the correct blocks are running
- Restart kernel before re-running code
- Explicit laser off command at the bottom of the page

In [10]:
import numpy as np
import pathlib
import pyvisa
import time
import matplotlib.pyplot as plt
import math
from datetime import datetime
import threading
import pandas as pd
import os

TODAY_STR = datetime.now().strftime("%d.%m.%Y") # Format the date as DD.MM.YYYY
print(f"\x1b[1;3;4;96mDate of running the code:\x1b[0m \x1b[1;3;4;92m{TODAY_STR}\x1b[0m")
print(f"Time of running the code: {datetime.now().strftime("%H:%M:%S")}")
print("\x1b[1;31mWait for it...\x1b[0m")

# Device communication - must always be run
rm = pyvisa.ResourceManager()
list_connections = rm.list_resources()      

# print(list_connections)
# Check device connection
if len(list_connections) != 0:
    print("Pyvisa running nominally")
else:
    raise ValueError("Connections not working")

Date of running the code: 26.11.2024
Time of running the code: 14:50:30
Wait for it...
Pyvisa running nominally


In [12]:
# # Laser connection. Laser is TSL
is_connect_success = False 
while not is_connect_success:
    try:
        TSL = rm.open_resource("TCPIP::169.254.82.30::5000::SOCKET", read_termination="\r")
        print(TSL.query("*IDN?"))
        print("\x1b[0;92mTSL Connection established\x1b[0m")
        is_connect_success = True
    except pyvisa.VisaIOError:
        print("Retrying...")
        time.sleep(0.5)

SANTEC,TSL-570,24070004,0026.0026.0011
TSL Connection established


In [13]:
# # Set parameters for wavelength sweep
WAV_START = 1536.3          # units of nm
WAV_END = 1536.6            # units of nm
speed = 1                   # Sweep speed (units of nm/s). Make as low as possible for 20MHz LPF
# delay_between_sweeps = 0.3        # Not relevant since we are setting to manual triggering rather auto

t_end = (WAV_END - WAV_START) / speed

TSL.write(":TRIG:OUTP 2")     # Set electronic trigger to trigger at the beginning of the sweep
TSL.write(':TRIG:INP:STAN 1')       # Turn on manual trigger mode
time.sleep(0.1)

print("\x1b[0;92mLaser settings configured\x1b[0m") 

Laser settings configured


In [19]:
# # PulseStreamer connection - for permanent AOM on
# import API classes into the current namespace
from pulsestreamer import PulseStreamer, Sequence

# Connect to Pulse Streamer
ip = '169.254.8.2'  # Do not change this!!
ps = PulseStreamer(ip)

# This is just for the AOM to be like it's not even there if set to True
perpetual_on = False     

# Create a sequence object
sequence = ps.createSequence()

# # on-off sequence creation
# Each time unit is 10**(-9) seconds
# Example: A pulse with 10µs (10000 units) HIGH (or 1) and 30µs (30000 units) LOW (or 0) levels

# Time period of one cycle in seconds, e.g. 1*10**(-6) is one microsecond
T = 0.05         # modify this according to what I want

# Duty ratio (fraction of on-time during one cycle). A float from 0 to 1.
D = 0.3         # modify this according to what I want

if perpetual_on is True:
    D = 1
    # print()
    print("\x1b[1;31mThe AOM is permanently on, so it's like it's not there\x1b[0m")
else:
    print(f"\x1b[1;31mThe AOM has a duty cycle {D} and time period of {T}s\x1b[0m")
    
if D > 1 or D < 0:
    raise ValueError("D must be between 0 and 1, inclusive.")

# Convert T into time units
pattern_T = math.ceil(T / (10**(-9)))
on_time = math.ceil(pattern_T * D)
off_time = math.ceil(pattern_T * (1-D))

pattern = [(on_time, 1), (off_time, 0)]

# Create sequence and assign pattern to digital channel 0
sequence.setDigital(0, pattern)

## Pattern implementation
# Stream the sequence and repeat it indefinitely
n_runs = PulseStreamer.REPEAT_INFINITELY
ps.stream(sequence, n_runs)

Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 169.254.8.2
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0
The AOM has a duty cycle 0.3 and time period of 0.05s


In [ ]:
# # oscilloscope connection
# Open the connection to the oscilloscope using its IP address
oscilloscope = rm.open_resource('TCPIP::169.254.112.67::INSTR', open_timeout=5000) # For the RIGOL DHO1204 in A*STAR


# Send a command to verify connection (e.g., identify the instrument)
# Sanity check for successful communication with oscilloscope
response = oscilloscope.query('*IDN?')
print(response, end='')
print("\x1b[0;92mOscilloscope Connection established.\x1b[0m")

In [ ]:
# # Oscilloscope configure settings
# Turn off roll for the trigger to work
oscilloscope.write(':TIMebase:ROLL 0')      


# # Sampling rate control
oscilloscope.write(':ACQuire:MDEPth 25M')    
oscilloscope.write(':ACQuire:TYPE NORM')      # Either NORMal or ULTRa, but ULTRa is limited in capability

oscilloscope.write(':CHANnel2:BWLimit 20M')     # Clean up the high frequency noise
oscilloscope.write(':CHANnel2:BWLimit ON') 


# Time scale - this has an effect on sampling!
buffer = 0.005      # 5ms buffer to catch full sweep
timescale = (t_end + buffer * 5) / 5
oscilloscope.write(f':TIMebase:MAIN:SCALe {timescale}')      # This is limited to certain values! Check manual!
oscilloscope.write(':TIMebase:MAIN:OFFSet 0')


# # Amplitude scale  - this has an effect on sampling!
oscilloscope.write(':CHANnel1:SCALe 3')         
oscilloscope.write(':CHANnel1:OFFSet 0')

oscilloscope.write(':CHANnel2:SCALe 0.5e-3')          # these ones need to be very carefully set!
oscilloscope.write(':CHANnel2:OFFSet -1e-3')      # these ones need to be very carefully set!


# # Display of waveforms
oscilloscope.write(':CHANnel1:DISPlay 1')       # Turn on display of channel 1
oscilloscope.write(':CHANnel2:DISPlay 1')       # Turn on display of channel 2

sampling_rate = float(oscilloscope.query(':ACQuire:SRATe?').strip())
if sampling_rate >= 1e9:
    formatted_rate = f"{sampling_rate / 1e9:.2f} GSa per s"
elif sampling_rate >= 1e6:
    formatted_rate = f"{sampling_rate / 1e6:.2f} MSa per s"
else:
    formatted_rate = f"{sampling_rate:.2f} Sa per s"

print(f"Acquisition mode is set to {oscilloscope.query(':ACQuire:TYPE?').strip()}")
print(f"Filter is set to {oscilloscope.query(':CHANnel2:BWLimit?').strip()}")
print(f"Sampling rate is currently to {formatted_rate} with the other settings in place")

print("\x1b[1;31mCheck or query settings again. Rerun cell if wrong.\x1b[0m")

Code below is for continuous looping of automatic triggers. It has now become VERY long because of a lot of simultaneous loop processes that occur at various instances in the loop, including laser control and plotting. The bulk of the code is in here. 

In [ ]:
threshold = 1 # Voltage threshold for triggering

def setup_trigger(trigger_level, trigger_source='CHAN1'):
    """Function to set up and enable triggering"""
    # Set the trigger source (e.g., Channel 1)
    oscilloscope.write(f':TRIGger:EDGE:SOURce {trigger_source}')
    # Set the trigger level (e.g., 0.5V)
    oscilloscope.write(f':TRIGger:EDGE:LEVel {trigger_level}')
    # Set the trigger type (e.g., edge trigger on a rising edge)
    oscilloscope.write(':TRIGger:EDGE:SLOPe POSitive')
    # Set the trigger sweep mode
    oscilloscope.write(':TRIGger:SWEep SINGle')
    # print(f"Trigger set to {trigger_level}V on {trigger_source}.")


def plot_and_save_waveform(channels: list[int]):
    """Function to save the waveform as CSV file. """
    
    for ch in channels:
        # Select the channel
        oscilloscope.write(f':WAV:SOUR CHAN{ch}')

        # Set the waveform format to ASCII (can also use 'BYTE' or 'WORD' for binary)
        oscilloscope.write(':WAV:FORM ASCII')

        # Get the waveform data
        print(f"Querying waveform data for channel {ch}")
        data = oscilloscope.query(':WAV:DATA?')

        # Parse the data (it's returned as a comma-separated string in ASCII mode)
        waveform_data = np.array([float(i) for i in data.split(',')])
        waveforms[f"waveform_ch{ch}"] = waveform_data

        # Get the X-axis scale and position (Time per division, etc.)
        x_increment = float(oscilloscope.query(':WAV:XINC?'))
        x_origin = float(oscilloscope.query(':WAV:XOR?'))

        # Generate the time axis
        time_axis = np.linspace(x_origin, x_origin + x_increment * len(waveform_data), len(waveform_data))
        waveforms[f"time_axis_ch{ch}"] = time_axis

        # Generate a timestamp for the file name
        timestamp = datetime.now().strftime("%d.%m.%Y_%H.%M.%S")
        # Generate a datestamp for the overall folder name
        datestamp = datetime.now().strftime("%d.%m.%Y")
        
        # Define the folder path
        output_folder = f"C:/Users/groov/VSCode/CSV_Waveforms/{datestamp}/{datestamp}_base_temp_power_sweeps"
        # Create the folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        
        if ch == 1:
            csvfilename = os.path.join(output_folder, f"{timestamp}_triggers_ch{ch}.csv")
            triggercsvfilenames.append(csvfilename)
        elif ch == 2:
            csvfilename = os.path.join(output_folder, f"{timestamp}_optical_ch{ch}_samplingrate_{formatted_rate}.csv")
            opticalcsvfilenames.append(csvfilename)

        # Save the waveform data to a CSV file
        np.savetxt(csvfilename, np.column_stack((time_axis, waveform_data)), delimiter=",", header="Time, Amplitude")
        
    #  # Plot the waveform for visualization and sanity check
    # plt.figure(figsize=(10, 6))
    # for ch in channels:
    #     plt.plot(waveforms[f"time_axis_ch{ch}"],  waveforms[f"waveform_ch{ch}"], label=f"channel {ch}")
    # plt.title("Waveform from Channels " + ", ".join(str(ch) for ch in channels))
    # plt.xlabel("Time (s)")
    # plt.ylabel("Amplitude (V)")
    # plt.grid(True)
    # plt.legend()
    # # plt.savefig(f"waveform_channel_{channel}_{timestamp}.png")
    # plt.show()

def load_data(file_path):
    """Load data from a CSV file."""
    data = pd.read_csv(file_path)
    time_data = data.iloc[:, 0].values
    signal_data = data.iloc[:, 1].values
    return time_data, signal_data


def find_first_trigger(signal_data, threshold=threshold):
    """Locate the trigger that happens at the start of a sweep"""
    trigger_indices = np.where(signal_data > threshold)[0]
    return trigger_indices[0]

print("Functions defined")

In [ ]:
# OLd trigger catching code (commented out)

# def find_first_trigger(time_data, signal_data, threshold=threshold, quiet_period_duration=quiet_period_duration):
#     """Find the first valid trigger based on the threshold and quiet period."""
#     # Identify indices where the signal exceeds the threshold
#     trigger_indices = np.where(signal_data > threshold)[0]
#     # print(f"Trigger indices (signal > {threshold}): {trigger_indices}")
    
#     for i in range(len(trigger_indices) - 1):
#         if time_data[trigger_indices[i+1]] - time_data[trigger_indices[i]] >= quiet_period_duration:
#             return trigger_indices, trigger_indices[i+1], time_data[trigger_indices[i+1]]

#     raise ValueError("No valid trigger found")


# def find_last_trigger(time_data, trigger_indices, valid_trigger_index, quiet_period_duration=quiet_period_duration):
#     """Find the last valid trigger within the same sweep as the first valid trigger. """

#     # Filter triggers to only those occurring after the first valid trigger (t=0)
#     post_t0_trigger_indices = trigger_indices[trigger_indices >= valid_trigger_index]
#     # print(f"Post t=0 trigger indices: {post_t0_trigger_indices}")

#     for i in range(len(post_t0_trigger_indices) - 1):
#         # Check if the time difference satisfies the quiet period condition
#         if time_data[post_t0_trigger_indices[i + 1]] - time_data[post_t0_trigger_indices[i]] >= quiet_period_duration:
#             # print(f"Last valid trigger found at index {post_t0_trigger_indices[i]} with time {time_data[post_t0_trigger_indices[i]]}s.")
#             return post_t0_trigger_indices[i], time_data[post_t0_trigger_indices[i]]

#     # If no valid last trigger is found, the last trigger in the dataset is valid
#     last_trigger_index = post_t0_trigger_indices[-1]
#     print(f"Defaulting to the last trigger in the dataset at index {last_trigger_index} with time {time_data[last_trigger_index]}s.")
#     return last_trigger_index, time_data[last_trigger_index]

In [ ]:
# TSL.write(":WAVelength:SWEep 0")  # Engineering reset
# time.sleep(0.1)
# TSL.write(":POWer:STATe 1")
# time.sleep(0.1)
# TSL.write(f":POW 6")
# time.sleep(0.1)
# TSL.write(":WAVelength:SWEep:MODe 1")   # Continuous and one-way sweep, for finest wavelength change per unit time.
# time.sleep(0.1)
# TSL.write(f":WAVelength:SWEep:SPEed {speed}")  
# time.sleep(0.1)
# TSL.write(":WAVelength:SWEep 1")      # Begin sweeping   
# time.sleep(0.1)

# print(TSL.query(':TRIG:INP:STAN?'))
# time.sleep(0.1)

# TSL.write(':TRIGger:INPut:SOFTtrigger')     # manual triggering
# time.sleep(0.1)

In [ ]:
# # # Test laser activation block
# TSL.write(f":WAVelength:SWEep:STARt {WAV_START}E-9") # Set start wavelength
# time.sleep(1)
# TSL.write(f":WAVelength:SWEep:STOP {WAV_END}E-9")   # Set stop wavelength
# time.sleep(1)
# TSL.write(f":WAVelength:SWEep:SPEed {speed}")  
# time.sleep(1)
# TSL.write(":WAVelength:SWEep:MODe 1")   # Continuous and one-way sweep, for finest wavelength change per unit time.
# time.sleep(1)
# TSL.write(":WAVelength:SWEep 1")      # Turn on sweep mode   
# time.sleep(1)
# TSL.write(':TRIG:INP:STAN 1')       # Turn on manual trigger mode
# time.sleep(1)
# print(TSL.query(':WAV:SWE?'))

# print("standing by")
# time.sleep(5)

In [ ]:
# # Data collection of optical absorption

waveforms = {}      # Initialise empty dictionary of waveforms

for set_laser_power in range(7, 13+1):      # Set laser power in dBm for range. Up to but not including!
    # Turn on laser to begin conducting sweeps
    TSL.write(":POWer:STATe 1")
    time.sleep(0.1)

    # The ten files that get saved here will be cleared at the end of the larger loop's long sleep.
    triggercsvfilenames = []
    opticalcsvfilenames = []

    print(f"Trigger set to {threshold}V on CHAN1.")
    
    TSL.write(f":POW {set_laser_power}")
    print(f"Laser is on, power set to {set_laser_power}dBm")
    time.sleep(0.1)      # time to check that the laser power is set correctly

    # # Settings implementation 
    TSL.write(":WAVelength:SWEep 0")  # Engineering reset
    time.sleep(0.1)
    TSL.write(":WAVelength:SWEep 1")      # Turn on sweep mode   
    time.sleep(0.1)

    TSL.write(f":WAVelength:SWEep:STARt {WAV_START}E-9") # Set start wavelength
    time.sleep(0.1)
    TSL.write(f":WAVelength:SWEep:STOP {WAV_END}E-9")   # Set stop wavelength
    time.sleep(0.1)
    TSL.write(f":WAVelength:SWEep:SPEed {speed}")  
    time.sleep(0.1)
    TSL.write(":WAVelength:SWEep:MODe 1")   # Continuous and one-way sweep, for finest wavelength change per unit time.
    time.sleep(0.1)
    

    for i in range(10):     # 10 captures
        print(f"Capture {i+1}/10")
        
        oscilloscope.write(':STOP') # Quickly "engineering reset" the oscilloscope
        time.sleep(0.1) # Small delay to let oscilloscope settle before running and arming trigger

        oscilloscope.write(':RUN') # Need oscilloscope to start running before it can capture anything
        # The trigger will automatically start capturing once the trigger is detected

        # # Arm the trigger, Unit of trigger_level is V
        # # Note that we run on single mode
        setup_trigger(trigger_level=threshold)  
        time.sleep(0.1) # Small delay to confirm trigger preparation

        # # Outer trigger - redundant
        # TSL.write(':TRIGger:INPut:SOFTtrigger')     # manual triggering
        # time.sleep(0.1)   
        # print("'Manually' triggered")      
        
        while 1:
            time.sleep(1) 
            if oscilloscope.query(':TRIGger:STATus?').strip() == 'STOP':
                print("Stopped capture, post single trigger")
                plot_and_save_waveform(channels=[1,2])   # TODO: Write in the list of channel numbers that we wish to plot and save
                print("Frozen waveforms captured and saved as CSV")
                # time.sleep(10)      # Some time to observe
                break
            elif oscilloscope.query(':TRIGger:STATus?').strip() == 'WAIT':
                TSL.write(':TRIGger:INPut:SOFTtrigger')     # manual (re)triggering
                time.sleep(0.1)
                print("'Manually' triggered")
                continue
            elif oscilloscope.query(':TRIGger:STATus?').strip() == 'TD':
                print("Caught a trigger!")
                continue
            else:       # For any other possible trigger status (AUTO, RUN), we want to loop until we hit a "wait", then send in a trigger
                time.sleep(0.1)  
                pass

    # ######## Out of the for-loop with 10 cycles of scanning, now need to process the raw data of those 10 sweeps ##########

    # Initialisation - store interpolated signals for averaging
    common_wavelength = None
    interpolated_signals = []
    
    for idx, (trigger_file, optical_file) in enumerate(zip(triggercsvfilenames, opticalcsvfilenames)):

        # Load trigger data
        trigger_time, trigger_signal = load_data(trigger_file)

        # Find the first valid trigger and mark t=0
        trigger_index = find_first_trigger(signal_data=trigger_signal)

        t0_reference = trigger_time[trigger_index]

        # Load optical data
        optical_time, optical_signal = load_data(optical_file)

        # Align optical data to t=0 from trigger data
        adjusted_optical_time = optical_time - t0_reference

        # Filter optical data to include only times between t=0 and t_end 
        valid_optical_indices = (adjusted_optical_time >= 0) & (adjusted_optical_time <= t_end)

        filtered_optical_time = adjusted_optical_time[valid_optical_indices]

        filtered_optical_signal = optical_signal[valid_optical_indices]

        # Convert filtered optical time to wavelength
        filtered_wavelength = WAV_START + speed * filtered_optical_time

        # Plot the filtered optical data 
        plt.figure(figsize=(10, 6))
        plt.plot(filtered_wavelength, filtered_optical_signal, label="Filtered Optical Data")
        plt.xlabel("Wavelength (nm)")
        plt.ylabel("Optical Signal Amplitude (V)")
        plt.title(f"Dataset {idx + 1}: Optical Data Filtered by Trigger Time Range")
        plt.legend()
        plt.grid()
        plt.show()

        interp_fineness = 1*10**5  # Number of points to interpolate. More points = finer grid and more info, but less accurate

        # Create a common wavelength axis if not already created
        if common_wavelength is None:
            common_wavelength = np.linspace(WAV_START, WAV_END, interp_fineness)  

        # Interpolate the signal to the common wavelength axis
        interpolated_signal = np.interp(common_wavelength, filtered_wavelength, filtered_optical_signal)
        # np.interp only operates on the specific inputs you provide:
        # New x-values where interpolation is desired (common_wavelength).
        # Known x-values (filtered_wavelength).
        # Corresponding known y-values (filtered_optical_signal).
        interpolated_signals.append(interpolated_signal)

    # Convert to NumPy array
    interpolated_signals = np.array(interpolated_signals)

    # Compute the average signal from the interpolated signals from common wavelength axis
    average_signal = np.mean(interpolated_signals, axis=0)

    # Plot the averaged optical wave plot
    plt.figure(figsize=(10, 6))
    plt.plot(common_wavelength, average_signal, label="Average Optical Signal")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Optical Signal Amplitude")
    plt.title(f"Averaged Optical Wave Plot at {set_laser_power}dBm")
    plt.legend()
    plt.grid()
    plt.show()  
    
    datestamp = datetime.now().strftime("%d.%m.%Y")
    timestamp = datetime.now().strftime("%d.%m.%Y_%H.%M.%S")

    # Define the folder path
    output_folder = f"C:/Users/groov/VSCode/CSV_Waveforms/{datestamp}/{datestamp}_base_temp_power_sweeps/final_optical_data"
    # Create the folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    csvfilename = os.path.join(output_folder, f"{set_laser_power}dBm_finalised_sweep_{timestamp}.csv")
    np.savetxt(csvfilename, np.column_stack((common_wavelength, average_signal)), delimiter=",",header="Wavelength (nm), Optical Signal (V)")
    
    # Reset sweeping after data has been collected
    TSL.write(":WAVelength:SWEep 0")
    time.sleep(0.1)
    TSL.write(":POWer:STATe +0")
    time.sleep(0.1)
    if set_laser_power != 13:
        print("Laser off, waiting for next power sweep in 5 minutes")
        time.sleep(1*60)    # 5 minutes needed to create a distinguishable time difference between data sets
    elif set_laser_power == 13:
        print("13dBm upper limit. Laser off, experiment concluded.")
        break


# # Explicit laser turn off once sweeps are over, just in case
time.sleep(0.1)
TSL.write(":WAVelength:SWEep 0")
time.sleep(0.1)
TSL.write(":WAVelength 1550nm")
time.sleep(0.1)
TSL.write(":POWer:STATe +0")
time.sleep(0.1)

In [ ]:
# # Explicit laser turn off
if TSL.query(":POWer:STATe?") != "+0":
    TSL.write(":POWer:STATe +0")
    time.sleep(0.1)